In [1]:
import h5py
import re
import hickle as hkl
import torch
import torch.nn as nn
from torch.legacy.nn import Reshape
import graphviz
import torch.nn.functional as F
from torch.autograd import Variable
from visualize import make_dot
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torch.utils.data as utils
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import imresize, imread, imshow
import time
import logging
from math import log,sqrt

In [ ]:
torch.cuda.current_device()

In [2]:
dense161 = models.densenet161(pretrained=True)
#dense161

In [ ]:
model5

In [ ]:
for param in model5.parameters():
    print param.size()

In [ ]:
c1 = 0
for para in dense161.parameters():
    c1 += para.numel()
c1

In [ ]:
deconvolution2 = nn.Sequential(nn.BatchNorm2d(2208),nn.ReLU(),nn.ConvTranspose2d(2208,512,kernel_size=(5,5),stride=(2,2),padding=(1,1)),
                             nn.BatchNorm2d(512),nn.ReLU(),nn.ConvTranspose2d(512,256,kernel_size=(5,5),stride=(2,2),padding=(1,1)),
                             nn.BatchNorm2d(256),nn.ReLU(),nn.ConvTranspose2d(256,128,kernel_size=(5,5),stride=(2,2),padding=(1,1)),
                             nn.BatchNorm2d(128),nn.ReLU(),nn.ConvTranspose2d(128,64,kernel_size=(5,5),stride=(2,2),padding=(1,1)),
                             nn.BatchNorm2d(64), nn.ReLU(),nn.ConvTranspose2d(64,1,kernel_size=(5,5),stride=(2,2),padding=(1,1))
                             )

In [3]:
deconvolution1 = nn.Sequential(nn.BatchNorm2d(2208),nn.ReLU(),nn.Conv2d(2208,512,1),
                             nn.BatchNorm2d(512),nn.ReLU(),nn.ConvTranspose2d(512,256,kernel_size=(5,5),stride=(2,2),padding=(1,1)),
                             nn.BatchNorm2d(256),nn.ReLU(),nn.ConvTranspose2d(256,128,kernel_size=(5,5),stride=(2,2),padding=(1,1)),
                             nn.BatchNorm2d(128),nn.ReLU(),nn.ConvTranspose2d(128,64,kernel_size=(5,5),stride=(2,2),padding=(1,1)),
                             nn.BatchNorm2d(64), nn.ReLU(),nn.ConvTranspose2d(64,1,kernel_size=(5,5),stride=(2,2),padding=(1,1))
                              )#nn.BatchNorm2d(64),nn.ReLU(),nn.Conv2d(64,1,1)
                             #)

In [ ]:
deconvolution = nn.Sequential(nn.BatchNorm2d(2208),nn.ReLU(),nn.Conv2d(2208,512,1),nn.BatchNorm2d(512),nn.ReLU(),nn.Conv2d(512,256,1),
                             nn.BatchNorm2d(256),nn.ReLU(),nn.ConvTranspose2d(256,128,kernel_size=(5,5),stride=(2,2),padding=(1,1)),
                             nn.BatchNorm2d(128),nn.ReLU(),nn.ConvTranspose2d(128,64,kernel_size=(5,5),stride=(2,2),padding=(1,1)),
                             nn.BatchNorm2d(64),nn.ReLU(),nn.ConvTranspose2d(64,1,kernel_size=(5,5),stride=(2,2),padding=(1,1))
                             )

In [ ]:
for m in deconvolution1.modules():
    print type(m)

In [4]:
for m in deconvolution1.modules():
    if isinstance(m,nn.Conv2d):
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0,sqrt(2./n))
    elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.fill_(1)
        m.bias.data.zero_()
    elif isinstance(m, nn.ConvTranspose2d):
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0,sqrt(2./n))

In [ ]:
temp1 = nn.Sequential(*list(dense161.children())[:-1])
temp2 = temp1.children()

temp3 = nn.Sequential(*list(temp1.children())[0])
temp3
temp4 = nn.Sequential(*list(temp3.children())[0:12])
print(temp4)
temp4.forward(Variable(torch.randn(2,3,240,320))).size()

In [ ]:
b1 = nn.Sequential(*list(dense161.children())[:-1])
b1

In [5]:
class MyModel5(nn.Module):
    def __init__(self, pretrained_model):
        super(MyModel5, self).__init__()
        self.pretrained_model = nn.Sequential(*list(dense161.children())[:-1])
        self.deconv1 = deconvolution1
        
        #self.unpool1 = nn.UpsamplingBilinear2d(scale_factor=2)
        #self.unpool2 = nn.UpsamplingBilinear2d(size=(60,80))
        self.conv0 = nn.Conv2d(2208,512,1)
        #self.conv1 = nn.Conv2d(512,256,1)
        
        #self.tconv1 = nn.ConvTranspose2d(256,128,kernel_size=(5,5),stride=(2,2),padding=(1,1))
        #self.tconv2 = nn.ConvTranspose2d(128,64,kernel_size=(5,5),stride=(2,2),padding=(1,1))
        #self.tconv3 = nn.ConvTranspose2d(64,1,kernel_size=(5,5),stride=(2,2),padding=(1,1))
        
        #self.tconv2 = nn.ConvTranspose2d(128,64,3,stride=2,padding=0)
        #self.tconv3 = nn.ConvTranspose2d(64,1,3,stride=2,padding=0)
        #self.conv1 = nn.Conv2d(1024,512,1)
        #self.conv1 = nn.Conv2d(1024,512,5,padding=(2,2))
        #self.conv2 = nn.Conv2d(512,256,5,padding=(2,2))
        #self.conv3 = nn.Conv2d(256,128,5,padding=(4,2))
        #self.conv4 = nn.Conv2d(128,1,3,padding=(1,1))
        #self.conv5 = nn.Conv2d(128,1,3,padding=(1,1))
        #self.bnorm = nn.BatchNorm2d(1024)
        self.bnorm1 = nn.BatchNorm2d(2208)
        #self.bnorm2 = nn.BatchNorm2d(512)
        #self.bnorm3 = nn.BatchNorm2d(256)
        #self.bnorm4 = nn.BatchNorm2d(128)
        #self.bnorm5 = nn.BatchNorm2d(64)
        
    def unpool_conv(self,inp1,conv,unpool):
        #x = inp1
        x = unpool(inp1)
        x = F.relu(conv(x))
        return x
    
    def newconv(self,inp1,conv,bn1):
        size1 = inp1.size()[1]
        #print('inside newconv ',size1)
          
        x = bn1(inp1)
        x = F.relu(x)
        x = conv(x)
        return x
        
    def forward(self, x):
        x = self.pretrained_model(x)
        #print(x.size()[1])

        #x = self.newconv(x,self.conv0,self.bnorm1)
        #x = self.newconv(x,self.conv1,self.bnorm2)
        #x = self.newconv(x,self.tconv1,self.bnorm3)
        #x = self.newconv(x,self.tconv2,self.bnorm4)
        #x = self.newconv(x,self.tconv3,self.bnorm5)
        #x = self.unpool_conv(x,self.conv1,self.unpool1)
        #x = self.unpool_conv(x,self.conv2,self.unpool1)
        #x = self.unpool_conv(x,self.conv3,self.unpool1)
        #x = self.unpool_conv(x,self.conv4,self.unpool2)
        #x = self.unpool2(x)
        #x = self.conv4(x)
        
        x = self.deconv1(x)
        x = x.view(-1,127,175)
        
        return x

In [6]:
model5 = MyModel5(dense161)
model5.forward(Variable(torch.randn(2,3,240,320))).size()

torch.Size([2, 127, 175])

In [ ]:
model6 = MyModel5(dense161)
model6.forward(Variable(torch.randn(2,3,240,320))).size()

In [ ]:
for params in model5.parameters():
    print(params.requires_grad)

In [ ]:
sum1 = 0
        
print("Number of layers ---> ",len(list(model5.parameters())))
for params in model5.parameters():
    if params.requires_grad == True:
        sum1 += params.numel()
    
print("Total number of parameters ---> ",sum1)

In [7]:
#dataset loading part...

#file1 = h5py.File('data/make3dTraining.h5')
#file2 = h5py.File('data/make3dTest1.h5')
#file3 = h5py.File('data/make3dTest2.h5')

file1 = h5py.File('data/Make3DTrain.h5')
file2 = h5py.File('data/Make3DTest.h5')

xtrain1 = np.array(file1['xtrain1'],dtype=np.float32)
ytrain1 = np.array(file1['ytrain1'],dtype=np.float32)


xtest1 = np.array(file2['xtrain1'],dtype=np.float32)
ytest1 = np.array(file2['ytrain1'],dtype=np.float32)
#ytest1 = np.moveaxis(ytest1,[0,1,2],[2,1,0])
#xtest1 = np.moveaxis(xtest1,[0,1,2,3],[1,3,2,0])

print("xtest shape ",xtest1.shape)
print("ytest shape ",ytest1.shape)
print("xtrain shape ",xtrain1.shape)
print("ytrain shape ",ytrain1.shape)
#xtrain1 /= 255
#xtest1 /= 255


('xtest shape ', (134, 3, 240, 320))
('ytest shape ', (134, 127, 175))
('xtrain shape ', (400, 3, 240, 320))
('ytrain shape ', (400, 127, 175))


In [ ]:
file2 = h5py.File('data/nyuV2testdata240new.h5')
xtest1 = np.array(file2['xtrain1'],dtype=np.float32)
ytest1 = np.array(file2['ytrain1'],dtype=np.float32)

In [ ]:
xtrain1 = np.clip(xtrain1,0.0,255.0)
print("xtrain shape ",xtrain1.shape)
print("max ",np.min(xtrain1))
print("min ",np.max(xtrain1))

In [ ]:
xtrain1 /= 255
xtest1 /= 255

mean1 = [0.485,0.456,0.406]
std1 = [0.229,0.224,0.225]

#xtrain1[:,0,:,:] = (xtrain1[:,0,:,:]-mean1[0])/std1[0]
#xtrain1[:,1,:,:] = (xtrain1[:,1,:,:]-mean1[1])/std1[1]
#xtrain1[:,2,:,:] = (xtrain1[:,2,:,:]-mean1[2])/std1[2]

xtest1[:,0,:,:] = (xtest1[:,0,:,:]-mean1[0])/std1[0]
xtest1[:,1,:,:] = (xtest1[:,1,:,:]-mean1[1])/std1[1]
xtest1[:,2,:,:] = (xtest1[:,2,:,:]-mean1[2])/std1[2]

#print("max in xtrain ---> ",np.max(xtrain1))
#print("min in xtrain ---> ",np.min(xtrain1))
print("max in xtest  ---> ",np.max(xtest1))
print("min in xtest  ---> ",np.min(xtest1))

#print("xtrain shape ---> ",xtrain1.shape)
#print("ytrain shape ---> ",ytrain1.shape)
print("xtest shape ---> ",xtest1.shape)
print("ytest shape ---> ",ytest1.shape)

In [ ]:
np.max(xtrain1)

In [ ]:
#center crop  with flipped bits from left to right..fil1

#file1 = h5py.File('data/xnewtrain1.h5')
#file2 = h5py.File('data/ynewtrain1.h5')

#xtrain1 = np.array(file1['xtrain1'],dtype=np.float32)
#ytrain1 = np.array(file2['ytrain1'],dtype=np.float32)



#xtrainfinal = np.zeros((1590,3,240,320),dtype=np.float32)
#ytrainfinal = np.zeros((1590,60,80),dtype=np.float32)
#ytrainfinal = np.zeros((1590,63,87),dtype=np.float32)

#xtrainfinal[:795,:,:,:] = xtrain1
#xtrainfinal[795:,:,:,:] = np.fliplr(xtrain1)

#ytrainfinal[:795,:,:] = ytrain1
#ytrainfinal[795:,:,:] = np.fliplr(ytrain1)


#print("xtrainfinal shape --> ",xtrainfinal.shape)
#print("ytrainfinal shape --> ",ytrainfinal.shape)
print("max in xtrain ---> ",np.max(xtrain1))
print("min in xtrain ---> ",np.min(xtrain1))
print("max in xtest  ---> ",np.max(xtest1))
print("min in xtest  ---> ",np.min(xtest1))

xtrain1 /= 255
#xtrainfinal /= 255
xtest1 /= 255

mean1 = [0.485,0.456,0.406]
std1 = [0.229,0.224,0.225]

xtrain1[:,0,:,:] = (xtrain1[:,0,:,:]-mean1[0])/std1[0]
xtrain1[:,1,:,:] = (xtrain1[:,1,:,:]-mean1[1])/std1[1]
xtrain1[:,2,:,:] = (xtrain1[:,2,:,:]-mean1[2])/std1[2]

#xtrainfinal[:,0,:,:] = (xtrainfinal[:,0,:,:]-mean1[0])/std1[0]
#xtrainfinal[:,1,:,:] = (xtrainfinal[:,1,:,:]-mean1[1])/std1[1]
#xtrainfinal[:,2,:,:] = (xtrainfinal[:,2,:,:]-mean1[2])/std1[2]

xtest1[:,0,:,:] = (xtest1[:,0,:,:]-mean1[0])/std1[0]
xtest1[:,1,:,:] = (xtest1[:,1,:,:]-mean1[1])/std1[1]
xtest1[:,2,:,:] = (xtest1[:,2,:,:]-mean1[2])/std1[2]

print("max in xtrain ---> ",np.max(xtrain1))
print("min in xtrain ---> ",np.min(xtrain1))
print("max in xtest  ---> ",np.max(xtest1))
print("min in xtest  ---> ",np.min(xtest1))

print("xtrain shape ---> ",xtrain1.shape)
print("ytrain shape ---> ",ytrain1.shape)
print("xtest shape ---> ",xtest1.shape)
print("ytest shape ---> ",ytest1.shape)

In [ ]:
np.nan in xtrain2

In [ ]:
def train(model, loss, optimizer, x_val, y_val):
    x = Variable(x_val,requires_grad = False).cuda()
    y = Variable(y_val,requires_grad = False).cuda()
    
    optimizer.zero_grad()
    fx = model.forward(x)
    output = loss.forward(fx,y)
    output.backward()
    
    optimizer.step()
    
    return output.data[0]

In [ ]:
xtrain2 = torch.from_numpy(xtrain1).float()
xtest2 = torch.from_numpy(xtest1).float()
ytrain2 = torch.from_numpy(ytrain1).float()
ytest2 = torch.from_numpy(ytest1).float()

In [ ]:
for param in model5.parameters():
    print(param.requires_grad)

In [ ]:
#xtrainfinal2 = torch.from_numpy(xtrainfinal).float()
#ytrainfinal2 = torch.from_numpy(ytrainfinal).float()

ytest2.size()
#np.min(xtrain1[0])
#xtrainfinal2.size()

In [ ]:
#custom loss function.... this will be reverse Huber...

class BerhuLoss(nn.Module):
    def __init__(self):
        super(BerhuLoss, self).__init__()
        
    def forward(self,input, target):
        #target is the ground truth value....
        mask1 = target.le(70)
        mask1 = mask1.float()
        count1 = mask1.eq(0).sum().float()
        
        target = mask1 * target
        input = mask1 * input
        
        if count1 == 0.0:
            count1 = 111125.0
        
        
        diff = torch.abs(target-input)
        batch_size, lossval = 5, 0.0
        c = 0.2 * torch.max(diff)
        c1 = c.data[0]
        temp = diff > c1
        check1 = torch.prod(temp)
        
        if check1 == 0:
            #if all elements in diff matrix are not greater than scalar c.
            lossval = torch.sum(diff)
            lossval = lossval/count1
        else:
            temp4 = torch.pow(diff,2)
            d = torch.pow(c,2)
            temp4 = temp4.add(d.expand_as(temp4))
            lossval = torch.sum(temp4/(2*c))
            lossval = lossval/count1
        return lossval

In [ ]:
t1 = Variable(torch.Tensor(5,127,175))
t2 = Variable(torch.Tensor(5,127,175))
t1 = t1.cuda()
t2 = t2.cuda()

b = BerhuLoss()

time1 = time.time()
samp = b.forward(t1,t2)
time2 = time.time() - time1

print("time ",time2)
samp

In [ ]:
#custom loss function....

class SquaredL2Norm(nn.Module):
    def __init__(self):
        super(SquaredL2Norm, self).__init__()
        
    def forward(self,input, target):
        diff = torch.abs(target-input)
        #temp1 = torch.sqrt(diff**2)
        temp1 = torch.sqrt(torch.sum(diff**2))
        return temp1

In [ ]:
#custom loss function....

class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        
    def forward(self,input, target):
        mask = target.le(70)
        
        mask1 = mask.float()
        count1 = mask1.eq(0).sum().float()
        target = mask1 * target        
        input = mask1 * input
        
        if count1 == 0:
            count1 = 111125.0
        
        diff = torch.abs(target-input)
        temp1 = diff**2
        temp2 = torch.sum(temp1)
        temp2 = temp2/count1
        return torch.sqrt(temp2)

In [ ]:
def optim_scheduler2(model,optimizer,epoch, init_lr=0.01, lr_decay_epoch=12):
    #lr = init_lr * (0.1**(epoch // lr_decay_epoch))
    #lr = 0.01
    #if epoch % lr_decay_epoch == 0:
    #    print('LR is set to {}'.format(lr))  
    #if epoch>20 and epoch<51:
    #    lr = 0.001
    #elif epoch>=50 and epoch<=100:
    #   lr = 0.0001
    #else:
    #    pass
    
    model6 = MyModel5(dense161)
    model6 = model6.cuda()
    model6.load_state_dict(torch.load("modeltest.pth"))
    
    for params in optimizer.param_groups:
        lr1 = params['lr']
    
    lr = lr1/5
    
    print("New learning rate ---> ",lr)
    optimizer = optim.SGD(model6.parameters(), lr=lr, momentum=0.9)
    return optimizer,model6

In [ ]:
#this code adds augmentation done online..
from imgaug import augmenters as iaa

seq1 = iaa.Sequential([
        iaa.Fliplr(1)
])

seq2 = iaa.Sequential([
        iaa.Affine(rotate=(-5,5))
])

seq3 = iaa.Sequential([
        iaa.Multiply((0.8,1.2))
])

seq1 = seq1.to_deterministic()
seq2 = seq2.to_deterministic()
seq3 = seq3.to_deterministic()

In [ ]:
t1 = xtest1[0:5]
t4 = ytest1[0:5]
t1 = np.moveaxis(t1,[0,1,2,3],[0,3,1,2])
print("shape now ",t1.shape)
imshow(t1[1])
print(np.min(t1[1]))
t2 = seq1.augment_images(t1)
t6 = seq1.augment_images(t4)
imshow(t2[1])
imshow(t6[1])
print(np.min(t2[1]))
t1 == t2

In [ ]:
np.random.randint(4)

In [ ]:
model5 = nn.DataParallel(model5, device_ids=[0,1])

In [ ]:
model5.load_state_dict(torch.load("make3dreports/BerhuTrain_129.pth"))

In [ ]:
#MUST UNCOMMENT BELOW LINE...
model5 = model5.cuda()

#loading the model after the weights of epoch50.. to check what loss the model gives if lr is taken as 0.0001
optimizer = optim.SGD(model5.parameters(), lr=0.001, momentum=0.9)

#optimizer = optim.SGD(model5.deconv1.parameters(), lr=0.001, momentum=0.9)
#optimizer = optim.SGD(model5.parameters(), lr=0.001, momentum=0.9)
#criterion = SquaredL2Norm()

criterion = BerhuLoss()
#criterion = RMSELoss()

currepochloss = float('Inf')
epochs, n_examples, i, batch_size, flag = 150,400, 0, 5, 0
tester = 0
saveme = 1 
while i != epochs:
    since = time.time()
    cost, batchloss = 0.0, 0.0
    num_batches = n_examples/batch_size
    
    indices = np.random.permutation(400)
    #batchwise training starts here...
    for k in range(num_batches):
        since1 = time.time()
        
        xtrain3 = torch.FloatTensor(batch_size,3,240,320)
        ytrain3 = torch.FloatTensor(batch_size,127,175)
        
        for ind in range(batch_size):
            #ind1 = np.random.randint(0,5599)
            ind1 = np.random.randint(0,399)
            newxind = indices[ind1]
            xtrain3[ind,:,:,:] = xtrain2[newxind,:,:,:]
            ytrain3[ind,:,:] = ytrain2[newxind,:,:]
        
        guess = np.random.randint(4)
        
        if guess == 0:
            #print("here1")
            store1 = xtrain3.numpy()
            store2 = ytrain3.numpy()
            store1 = np.moveaxis(store1,[0,1,2,3],[0,3,1,2])
            out1 = seq1.augment_images(store1)
            out2 = seq1.augment_images(store2)
            out1 = np.moveaxis(out1,[0,1,2,3],[0,2,3,1])
            xtrain3 = torch.from_numpy(out1).float()
            ytrain3 = torch.from_numpy(out2).float()
        elif guess == 2:
            #print("here2")
            store1 = xtrain3.numpy()
            store2 = ytrain3.numpy()
            store1 = np.moveaxis(store1,[0,1,2,3],[0,3,1,2])
            out1 = seq2.augment_images(store1)
            out2 = seq2.augment_images(store2)
            out1 = np.moveaxis(out1,[0,1,2,3],[0,2,3,1])
            xtrain3 = torch.from_numpy(out1).float()
            ytrain3 = torch.from_numpy(out2).float()
        elif guess == 3:
            #print("here3")
            store1 = xtrain3.numpy()
            store2 = ytrain3.numpy()
            store1 = np.moveaxis(store1,[0,1,2,3],[0,3,1,2])
            out1 = seq3.augment_images(store1)
            out2 = seq3.augment_images(store2)
            out1 = np.moveaxis(out1,[0,1,2,3],[0,2,3,1])
            xtrain3 = torch.from_numpy(out1).float()
            ytrain3 = torch.from_numpy(out2).float()
            
        
            
        batchloss = train(model5,criterion, optimizer, xtrain3, ytrain3)
    
        batch_time = time.time() - since1
        cost += batchloss
        #str1 = 'Loss = {:.4f} for batch number = {:d} at epoch {:d} completed in {:.0f}m {:.0f}s'.format(batchloss,k,(i+1),(batch_time//60),(batch_time%60))
        #logger1.info(str1)
        #print('Loss = {:.4f} for batch number = {:d} at epoch {:d} completed in {:.0f}m {:.0f}s'.format(batchloss,k,(i+1),(batch_time//60),(batch_time%60)))
        #print("Loss for batch number "+str(k)+" is " +str(batchloss)+" and takes "+str(batch_time//60)+"m "+str(batch_time%60)+"s")
    time_elapsed = time.time() - since
    epochloss = cost/num_batches
    
    if epochloss < currepochloss:
        torch.save(model5.state_dict(),"make3dreports/Make3dBerhu.pth")
        flag = 0
        currepochloss = epochloss
    else:
        flag += 1
        
        if flag == 6:
            for p in optimizer.param_groups:
                lr2 = p['lr']
            newlr = lr2/5
            if newlr < 1e-7:
                print("Cant decrease further!!")
                newlr = 1e-7
            flag = 0 
            optimizer = optim.SGD(model5.parameters(), lr=newlr, momentum=0.9)
            print("Learning rate changed from "+str(lr2)+" to "+str(newlr))
            
        print("Loss "+str(epochloss)+" is bigger than Loss "+str(currepochloss)+" in the prev epoch ")
    
    #str2 = 'Loss = {:.4f} at epoch {:d} completed in {:.0f}m {:.0f}s'.format((cost/num_batches),(i+1),(time_elapsed//60),(time_elapsed%60))
    #logger2.info(str2)
    print('Loss = {:.4f} at epoch {:d} completed in {:.0f}m {:.0f}s'.format(epochloss,(i+1),(time_elapsed//60),(time_elapsed%60)))
    i += 1 

    testchecker()
    torch.save(model5.state_dict(),"make3dreports/newreport"+str(saveme)+".pth")
    saveme += 1
        

In [ ]:
#torch.save(model5.state_dict(),"RMSE5974withRMSE3524samp-lrnow001-10to20to50to30epochs.pth")
torch.save(model5.state_dict(),"RMSE5748epochs120-140Berhu.pth")

In [ ]:
model6 = MyModel5(dense161)
model6 = nn.DataParallel(model6, device_ids=[0,1])
#model6.load_state_dict(torch.load("pth_Weights/RMSE5748epochs120-140Berhu.pth"))
#model6.load_state_dict(torch.load("data/Berhu8724samples_test_7.pth"))
model6.load_state_dict(torch.load("make3dreports/BerhuTrain_129.pth"))

In [ ]:
for params in optimizer.param_groups:
    print params['lr']

In [ ]:
def rmse_calc2(ytrue, ypred):
    mask = ytrue.le(70)
    mask1 = mask.float()
    count1 = mask1.eq(0).sum().float()
    ytrue = mask1 * ytrue        
    ypred = mask1 * ypred
        
    if count1 == 0.0:
        count1 = 111125.0
        #count1 = 44450
        
    diff = torch.abs(ytrue-ypred)
    temp1 = diff**2
    temp2 = torch.sum(temp1)
    temp2 = temp2/count1
    return torch.sqrt(temp2)

In [ ]:
#rmse calculation function..

def rmse_calc(ytrue, ypred):
    mask = ytrue.le(70)
    mask1 = mask.float()
    count1 = mask1.eq(0).sum().float()
    ytrue = mask1 * ytrue        
    ypred = mask1 * ypred
        
    if count1 == 0.0:
        #count1 = 111125.0
        count1 = 44450
        
    diff = torch.abs(ytrue-ypred)
    temp1 = diff**2
    temp2 = torch.sum(temp1)
    temp2 = temp2/count1
    return torch.sqrt(temp2)

In [ ]:
def testchecker():

    #testing of the architecture...
    test_loss = 0
    rmse_loss = 0
    num_batches = 0
    curr_rmse=0

    #xtest1 = xtest1.cuda()
    #ytest1 = ytest1.cuda()

    #6 evenly divides the test batch size..
    test_batch_size = 2
    test_epochs = 1
    n_examples = 134
    #n_examples = 795
    count = 0
    array1 = [0]*67
    finalpred = Variable(torch.zeros((n_examples,127,175)))
    #print("finalpred size is ---> ", finalpred.size())

    for i in range(test_epochs):
        cost = 0
        num_batches = n_examples/test_batch_size
        #print("num of batches --->", num_batches)
        for k in range(num_batches):
            start, end = k*test_batch_size, (k+1)*test_batch_size
            output = model5.forward(Variable(xtest2[start:end], volatile=True).cuda())
            finalpred[start:end] = output
            #print("output size is ---> ",output.size()," count is --->",count, " start is -->", start, " end is -->", end)
            count += 1
            curr_rmse = rmse_calc(Variable(ytest2[start:end]).cuda(), output)
            array1[k] = curr_rmse.data[0]
            rmse_loss += curr_rmse
            #print("curr rmse = %f with batch number = %d" % (curr_rmse.data.cpu().numpy(),k))

        print("Test RMSE cost = %f", rmse_loss/num_batches)

In [ ]:
#testing of the architecture...
test_loss = 0
rmse_loss = 0
num_batches = 0
curr_rmse=0

#model6 = model6.cuda()

#xtest1 = xtest1.cuda()
#ytest1 = ytest1.cuda()

#6 evenly divides the test batch size..
test_batch_size = 2
test_epochs = 1
n_examples = 134
#n_examples = 795
count = 0
array1 = [0]*67
finalpred = Variable(torch.zeros((n_examples,127,175)))
print("finalpred size is ---> ", finalpred.size())

for i in range(test_epochs):
    cost = 0
    num_batches = n_examples/test_batch_size
    print("num of batches --->", num_batches)
    timer1 = time.time()
    for k in range(num_batches):
        start, end = k*test_batch_size, (k+1)*test_batch_size
        output = model5.forward(Variable(xtest2[start:end], volatile=True).cuda())
        finalpred[start:end] = output
        #print("output size is ---> ",output.size()," count is --->",count, " start is -->", start, " end is -->", end)
        count += 1
        curr_rmse = rmse_calc(Variable(ytest2[start:end]).cuda(), output)
        array1[k] = curr_rmse.data[0]
        rmse_loss += curr_rmse
        print("curr rmse = %f with batch number = %d" % (curr_rmse.data.cpu().numpy(),k))
    finish = time.time() - timer1   
    print("RMSE cost = %f", rmse_loss/num_batches)
    print("min --> ",finish//60)
    print("sec --> ",finish%60)


In [ ]:
d = torch.rand(2,2)

p = d.numpy()

#Variables are wrappers around tensors that save operation history. the data can accessed with below method
data1 = finalpred.data.numpy()
print(data1.shape)

#sxtest2 = xtest1.numpy()
#sytest2 = ytest1.numpy()

#print(sxtest2.shape)

In [ ]:
np.min(data1)

In [ ]:
print(data1[79][0][100])
print(ytest1[79][0][100])

In [ ]:
#to make the background go blue.... the indices where ground truth is more than 70, it can be replaced with 0
# and 0 in the prediction (data) for the same indices... should work....

In [ ]:
ytest5 = ytest1.copy()
inds2 = np.where(ytest5 > 70.0)
#print(inds2)
ytest5[inds2[0],inds2[1],inds2[2]] = 0.0
data1[inds2[0],inds2[1],inds2[2]] = 0.0
#print(ytest2[0])
ytest1[0]

#ytest2[ytest2 > 70.0] = 0.0
#ytest2

In [ ]:
#output display using matplotlib and bokeh... ONLY FOR TEST IMAGES

plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['image.interpolation'] = 'none'
%matplotlib inline

fig = plt.figure()
fig.set_figheight(9)
fig.set_figwidth(9)
ind = 0
flag = 0

#ytest3 = ytestmod.numpy()
ytest3 = ytest5
#imageinds = [79,80,81,82,83]
imageinds = [64,32,84,63,87]

for i in range(1,21,4):
    nind = imageinds[ind]
    img1 = np.zeros((240,320,3))
    temp = xtest1[nind]
    #print(finxtest1[5].shape)

    img1[:,:,0] = temp[0,:,:]
    img1[:,:,1] = temp[1,:,:]
    img1[:,:,2] = temp[2,:,:]
    img1 = imresize(img1,(240,320,3))
    
    a=fig.add_subplot(5,4,i)
    imgplot = plt.imshow(img1)
    if flag == 0:
        a.set_title('Input')
    a.axes.get_xaxis().set_visible(False)
    a.axes.get_yaxis().set_visible(False)
    a=fig.add_subplot(5,4,i+1)
    ##################################################
    imgplot = plt.imshow(imresize(data1[nind],(480,640)))
    imgplot.set_cmap('jet')
    #print("hi inside")
    if flag == 0:
        a.set_title('Predicted Depth')
    a.axes.get_xaxis().set_visible(False)
    a.axes.get_yaxis().set_visible(False)
    ##################################################
    a=fig.add_subplot(5,4,i+2)
    imgplot = plt.imshow(ytest3[nind])
    imgplot.set_cmap('jet')
    if flag == 0:
        a.set_title('Actual Depth')
    a.axes.get_xaxis().set_visible(False)
    a.axes.get_yaxis().set_visible(False)
    ##################################################
    a=fig.add_subplot(5,4,i+3)
    error1 = np.abs(data1[nind]-ytest3[nind])
    #print "max is --->", error1.max()
    #print "min is --->", error1.min()
    imgplot = plt.imshow(error1)
    if flag == 0:
        a.set_title('Absolute Error')
    #imgplot.set_rotation(0)
    a.axes.get_xaxis().set_visible(False)
    a.axes.get_yaxis().set_visible(False)
    #imgplot.set_cmap('hot')
    imgplot.set_cmap('jet')
    plt.colorbar(fraction=0.080, pad=0.03)
    #plt.colorbar(fraction=0.080, pad=0.03,ticks=[0.0,0.2,0.4,0.6])
    #plt.clim(0,1)
    plt.clim(vmin=0, vmax=0.7)
    ##################################################
    ind += 1
    flag = 1

In [ ]:
sample = ytest2[1]
mask1 = ytest2[1].le(70).float()
sample
mask1*sample
mask1.eq(0).sum()

In [ ]:
#absolute error

def abs_error(ytrue, ypred):
    #ytrue is the actual values of the depth map..
    #ypred is the predicted values of the depth map from the modified model..
    
    mask = ytrue.le(70)
    #print(mask)
    mask1 = mask.float()
    #print("type ",type(mask1))
    count1 = mask1.eq(0).sum()
    print("count is ",count1)
    ytrue = mask1 * ytrue        
    ypred = mask1 * ypred
    
    if count1 == 0.0:
        count1 = 2978150.0
    
    diff = torch.abs(ytrue-ypred)
    #print(diff)
    temp1 = diff/ytrue
    temp2 = torch.sum(temp1)
    temp2 = temp2/count1
    return temp2

def abs_error1(ytrue,ypred):
    #print("ytrue size ",ytrue.shape)
    inds = np.where(ytrue < 70.0)
    print(len(inds[0]))
    print(inds[0])
    #print("inds size ",inds.shape)
    ytrue = ytrue[inds[0],inds[1],inds[2]]
    ypred = ypred[inds[0],inds[1],inds[2]]
    rel = np.abs(ytrue-ypred)/ytrue
    rel = np.mean(rel)
    return rel

def sq_error(ytrue,ypred):
    diff = torch.abs(ytrue-ypred)
    diff = diff**2
    temp1 = diff/ytrue
    temp2 = torch.mean(temp1)
    return temp2

def log_rmse(ytrue,ypred):
    #print("type --> ",type(ytrue))
    inds = np.where(ytrue < 70.0)
    #print("inds ",inds)
    ytrue = ytrue[inds[0],inds[1],inds[2]]
    ypred = ypred[inds[0],inds[1],inds[2]]
    diff = abs(np.log10(ytrue)-np.log10(ypred))
    #temp1 = diff**2
    print("num elements ",len(ypred))
    temp2 = np.mean(diff)
    return temp2


In [ ]:
final1 = finalpred.data.numpy()
final1.shape

In [ ]:
ans1 = abs_error1(ytest1,data1)
print("abs error --> ",ans1)

print("type is ",type(ytest2))

ans3 = log_rmse(ytest1,data1)
print("log rmse error --> ",ans3)

#per1,per2,per3 = calc_threshold(ytest2,finalpred.data,654)

In [ ]:
print(per1)
print(per2)
print(per3)

In [ ]:
print(per1)
print(per2)
print(per3)

In [ ]:
from scipy.ndimage import zoom, rotate

def rand_aug(im,z):
    
    ang = np.float32(np.random.random(1)*10.0-5.0); ang = ang[0]
    print("angle is ---> ",ang)
    rgb = np.float32(np.random.random(3)*0.4+0.8);
    print("rgb value --> ",rgb)
    
    ctrst = np.float32(np.random.random(1)+0.5); ctrst = ctrst[0]
    print("ctrss --> ",ctrst)

    im = rotate(im,ang,reshape=False,mode='nearest',order=1).copy()
    z = rotate(z,ang,reshape=False,mode='nearest',order=1).copy()
    print("im shape ---> ",im.shape)

      
    for j in range(3):
        im[:,:,j] = im[:,:,j] * rgb[j]
    im[...] = np.minimum(255.0,(im[...] ** ctrst) * (255.0**(1.0-ctrst)))
    
    return im,z

In [ ]:
im,z = rand_aug(xtest1[0],ytest1[0])
np.min(im)
im[...].shape

In [ ]:


def color1(im):
    rgb = np.float32(np.random.random(3)*0.4+0.8)
    im1 = np.zeros((240,320,3),dtype=np.float32)
    for j in range(3):
            im1[:,:,j] = im[:,:,j] * rgb[j]
            
    ctrst = np.float32(np.random.random(1)+0.5); ctrst = ctrst[0]
    im1[...] = np.minimum(255.0,(im1[...] ** ctrst) * (255.0**(1.0-ctrst)))
    return im1

In [ ]:
t1 = xtest1

In [ ]:
sample1 = xtest1[0]
#hold = np.zeros((240,320,3))
print("sample shape --> ",sample1.shape)
samp1 = np.moveaxis(sample1,[0,1,2],[2,0,1])
print("new shape --> ",samp1.shape)
imshow(samp1)
im2 = color1(samp1)
imshow(im2)

In [ ]:
samp1 == im2

In [ ]:
import torch.utils.data as data_utils

In [ ]:
train1 = data_utils.TensorDataset(xtrain2,ytrain2)

In [ ]:
train_loader = data_utils.DataLoader(train1,batch_size=5,shuffle=True)

In [ ]:
for i, (i1,t1) in enumerate(train_loader):
    print(type(i1))
    temp1 = i1.numpy()
    print(temp1.shape)
    t = np.moveaxis(temp1,[0,1,2,3],[0,3,1,2])
    imshow(t[0])
    t[0] = np.fliplr(t[0])
    imshow(t[0])
    break

In [ ]:
from imgaug import augmenters as iaa
t = np.random.uniform(1.0,1.5)
print(t)
seq1 = iaa.Sequential([
    iaa.Fliplr(0.5),
    #iaa.Affine(scale=(t,t),rotate=(-5,5))
    #iaa.Multiply((0.8,1.2))

])
seq = seq1.to_deterministic()

In [ ]:
new3 = np.moveaxis(xtest1,[0,1,2,3],[0,3,1,2])
new1 = seq.augment_images(new3)
new2 = seq.augment_images(ytest1)

In [ ]:
ht1 = xtest1[1]
ht1 = np.moveaxis(ht1,[0,1,2],[2,0,1])
imshow(ht1)
imshow(new1[1])
imshow(new2[1])

In [ ]:
#new2[0] = ytest1[0]/1.4933
print(new2[0])
dep1 = new2[0].copy()/1.4933
print("depth here")
print(dep1)
imshow(dep1)

In [ ]:
dep1[126][85]